## Genetic Programming with Energy Data

Data from the [National Grid ESO API ](https://www.nationalgrideso.com/data-portal/api-guidance). 

In [1]:
# imports and installs

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
import random
import itertools
import sys

from deepdiff import DeepDiff

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import HDBSCAN
from sklearn_extra.cluster import KMedoids
from scipy.cluster.hierarchy import linkage
from scipy.cluster.hierarchy import fcluster
from scipy.cluster.hierarchy import dendrogram
from scipy.spatial import distance

from sklearn.metrics import silhouette_score
from sklearn.metrics import davies_bouldin_score

## Description of Terms
Note: Units for flow are measured in MW. The following terms are descriptions of the time series. There are additional time series included which represent the carbon intensity forecast for 2023 of various regions. 

- IFA_FLOW: IFA stands for Interconnexion France-Angleterre, which is a subsea electricity link between France and Great Britain that began operating in 1986. It's a joint venture between National Grid and the French Transmission Operator RTE. The system-to-system flow (SSF) of the IFA is calculated and adjusted for interconnector losses to determine the flow. 
- TSD: This is the Transmission System generation requirement and is equivalent to the Initial Transmission System Outturn (ITSDO) and Transmission System Demand Forecast on BM Reports. Transmission System Demand is equal to the ND plus the additional generation required to meet station load, pump storage pumping and interconnector exports.
- VIKING_FLOW: Flow coming from a record-breaking 475-mile-long land and subsea cable connecting British and Danish energy grids for the first time.
- IFA2_FLOW: Commissioned in 2021 IFA2 is a 1,000 MW high voltage direct current (HVDC) electrical interconnector between the British and French transmission systems. It is the second link to France that National Grid has developed with RTE.
- EMBEDDED_WIND_GENERATION: This is an estimate of the GB wind generation from wind farms which do not have Transmission System metering installed. These wind farms are embedded in the distribution network and invisible to National Grid. Their effect is to suppress the electricity demand during periods of high wind. The true output of these generators is not known so an estimate is provided based on National Grid’s best model.
- ND: This is the Great Britain generation requirement and is equivalent to the Initial National Demand Outturn (INDO) and National Demand Forecast as published on BM Reports. National Demand is the sum of metered generation, but excludes generation required to meet station load, pump storage pumping and interconnector exports. National Demand is calculated as a sum of generation based on National Grid operational generation metering. 
- MOYLE_FLOW: Flow related to The Moyle Interconnector, a 500 megawatt (MW) HVDC link between Scotland and Northern Ireland, running between Auchencrosh in Ayrshire and Ballycronan More in County Antrim. It went into service in 2001
- NEMO_FLOW: Flow from Nemo Link, a 1,000 MegaWatt HVDC submarine power cable between Richborough Energy Park in Kent, the United Kingdom and Zeebrugge, Belgium
- ELECLINK_FLOW: Flow from ElecLink, a 1,000 MW high-voltage direct current (HVDC) electrical interconnector between the United Kingdom and France, passing through the Channel Tunnel.
- PUMP_STORAGE_PUMPING: The demand due to pumping at hydro pump storage units; the -ve signifies pumping load.
- EMBEDDED_WIND_CAPACITY: This is National Grid’s best view of the installed embedded wind capacity in GB. This is based on publically available information compiled from a variety of sources and is not the definitive view. It is consistent with the generation estimate provided above. 
- SETTLEMENT_DATE: Settlement Date. 
- ENGLAND_WALES_DEMAND: England and Wales Demand, as ND above but on an England and Wales basis.
- EMBEDDED_SOLAR_CAPACITY: As embedded wind capacity above, but for solar generation.
- SCOTTISH_TRANSFER: Power transfer across the scottish boundaries due to growth in renewable generation capacity.
- NON_BM_STOR: Operating reserve for units that are not included in the ND generator definition. This can be in the form of generation or demand reduction.
- SETTLEMENT_PERIOD: Settlement Period. 
- EAST_WEST_FLOW: Flow from the East - West Interconnector between Ireland and Great Britain.
- NSL_FLOW: Flow from the North Sea Link (NSL), a joint venture with Norwegian system operator Statnett. Stretching 720 kilometres under the North Sea, at depths of up to 700 metres, NSL is an interconnector capable of sharing up to 1400 megawatts of electricity
- BRITNED_FLOW: Flow from the Britned connector, a two-way 1,000 MW high-voltage direct current connection has a length of 260 km and runs from the Isle of Grain (in Kent) to Maasvlakte (near Rotterdam)
- _ID: Line ID. 
- EMBEDDED_SOLAR_GENERATION: As embedded wind generation above, but for solar generation.

In [2]:
# API Calls to the Britain national grid API. Calling to retrieve historic electricity demand,
# interconnector, wind and solar outturn, and carbon intensity data for 2022 and/or 2023.

URL = 'https://api.nationalgrideso.com/api/3/action/datastore_search_sql?sql=SELECT * FROM "bb44a1b5-75b1-4db2-8491-257f23385006"'
response = requests.get(URL).json()
URL2 = 'https://api.nationalgrideso.com/api/3/action/datastore_search_sql?sql=SELECT * FROM "bf5ab335-9b40-4ea4-b93a-ab4af7bce003"'
response2 = requests.get(URL2).json()
URL3 = 'https://api.nationalgrideso.com/api/3/action/datastore_search_sql?sql=SELECT * FROM "3372646d-419f-4599-97a9-6bb4e7e32862"'
response3 = requests.get(URL3).json()
URL4 = 'https://api.nationalgrideso.com/api/3/action/datastore_search_sql?sql=SELECT * FROM "c16b0e19-c02a-44a8-ba05-4db2c0545a2a"'
response4 = requests.get(URL4).json()


# Converting responses from json into pandas dataframe
df_demand_2022 = pd.json_normalize(
    response["result"]["records"],
    meta=[
        "IFA_FLOW",
        "TSD",
        "VIKING_FLOW",
        "IFA2_FLOW",
        "EMBEDDED_WIND_GENERATION",
        "ND",
        "MOYLE_FLOW",
        "NEMO_FLOW",
        "ELECLINK_FLOW",
        "PUMP_STORAGE_PUMPING",
        "EMBEDDED_WIND_CAPACITY",
        "SETTLEMENT_DATE",
        "ENGLAND_WALES_DEMAND",
        "EMBEDDED_SOLAR_CAPACITY",
        "SCOTTISH_TRANSFER",
        "NON_BM_STOR",
        "_FULL_TEXT",
        "SETTLEMENT_PERIOD",
        "EAST_WEST_FLOW",
        "NSL_FLOW",
        "BRITNED_FLOW",
        "_ID",
        "EMBEDDED_SOLAR_GENERATION",
    ],
)

df_demand_2023 = pd.json_normalize(
    response2["result"]["records"],
    meta=[
        "IFA_FLOW",
        "TSD",
        "VIKING_FLOW",
        "IFA2_FLOW",
        "EMBEDDED_WIND_GENERATION",
        "ND",
        "MOYLE_FLOW",
        "NEMO_FLOW",
        "ELECLINK_FLOW",
        "PUMP_STORAGE_PUMPING",
        "EMBEDDED_WIND_CAPACITY",
        "SETTLEMENT_DATE",
        "ENGLAND_WALES_DEMAND",
        "EMBEDDED_SOLAR_CAPACITY",
        "SCOTTISH_TRANSFER",
        "NON_BM_STOR",
        "_FULL_TEXT",
        "SETTLEMENT_PERIOD",
        "EAST_WEST_FLOW",
        "NSL_FLOW",
        "BRITNED_FLOW",
        "_ID",
        "EMBEDDED_SOLAR_GENERATION",
    ],
)

df_historic_prices_2022 = pd.json_normalize(
    response3["result"]["records"],
    meta=[
        "Settlement Period",
        "Half-hourly Charge",
        "Run Type",
        "Total Daily BSUoS Charge",
        "_full_text",
        "BSUoS Price (£/MWh Hour)",
        "Settlement Day",
        "_id",
    ],
)

df_carbon = pd.json_normalize(
    response4["result"]["records"],
    meta=[
        "East Midlands",
        "East England",
        "West Midlands",
        "North Scotland",
        "South Scotland",
        "_full_text",
        "South West England",
        "datetime",
        "North Wales and Merseyside",
        "North East England",
        "South East England",
        "South Wales",
        "North West England",
        "Yorkshire",
        "London",
        "_id",
        "South England"
    ]
)

# Conversions to datetime for extracting data for specific years
df_historic_prices_2022["Settlement Day"] = pd.to_datetime(
    df_historic_prices_2022["Settlement Day"]
)
df_historic_prices_2022 = df_historic_prices_2022[
    df_historic_prices_2022["Settlement Day"].dt.year == 2022
]

df_carbon['datetime'] = pd.to_datetime(
    df_carbon['datetime']
)
df_carbon_2023 = df_carbon[
    df_carbon['datetime'].dt.year == 2023
]

# Dropping unused columns for future concatenation
df_demand_2022.drop(["_full_text", "NON_BM_STOR"], axis=1, inplace=True)
df_demand_2023.drop(["_full_text", "NON_BM_STOR"], axis=1, inplace=True)
df_historic_prices_2022.drop(["Run Type", "_full_text"], axis=1, inplace=True)
df_carbon_2023.drop(["_full_text"], axis=1, inplace=True)


/var/folders/vr/_3gy4clj6932_zwdwb7jml_m0000gn/T/ipykernel_10079/563482866.py:123: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_carbon_2023.drop(["_full_text"], axis=1, inplace=True)


In [3]:
# Dropping id columns and now unused date columns. 
# Only want the time series that will be clustered - each are of size 17520
# so the "_id" column is able to be dropped. 

df_demand_2022_noid = df_demand_2022.drop(
    ["_id", "SETTLEMENT_DATE", "SETTLEMENT_PERIOD"], axis=1
)
df_demand_2023_noid = df_demand_2023.drop(
    ["_id", "SETTLEMENT_DATE", "SETTLEMENT_PERIOD"], axis=1
)
df_demand_2023_noid.columns = [str(col) + "_2" for col in df_demand_2023_noid.columns]
df_historic_prices_2022_noid = df_historic_prices_2022.drop(
    ["Settlement Period", "Settlement Day", "_id"], axis=1
)
df_carbon_2023_noid = df_carbon_2023.drop(
    ["_id", "datetime"], axis=1
)

# Concatenating the dataframes. 
df_full = pd.concat(
    [
        df_historic_prices_2022_noid.reset_index().drop("index", axis=1, inplace=True),
        df_demand_2022_noid,
        df_demand_2023_noid,
        df_carbon_2023_noid.reset_index().drop("index", axis=1, inplace=True)
    ],
    axis=1,
)

# Must perform scaling since clustering algorithms works on similarity/distance
df_full = StandardScaler().fit_transform(df_full)
df_full_transposed = df_full.transpose()

Must add in a step to check for nulls and other errors

We are going to instantiate multiple clustering models as our initial population for the algorithm. This can include the following: 
- KMeans
    - n_clusters: 3, 4, 5, 6
    - max_iter: 100, 200, 300, 400
    - tol: 0.00001, 0.0001, 0.001, 0.01
- KMedoids
    - n_clusters: 3, 4, 5, 6
    - metric: euclidean, cosine, haversine, l2 
    - method: alternate, pam
    - max_iter: 100, 200, 300, 400
- DBSCAN
    - eps: 0.2, 0.5, 1.0
    - min_samples: 3, 5, 10
    - metric: euclidean, cosine, haversine, l2 
- HDBSCAN
    - metric: euclidean, cosine, haversine, l2 
    - min_samples: 3, 5, 10
    - cluster_selection_epsilon: 0.2, 0.5, 1.0


In [4]:
# Defining which parameters are appropriate to adjust for each clustering model

model_list = ["KMeans", "KMedoids", "DBSCAN", "HDBSCAN"]

list_dict_model_params = [
    {"KMeans": ["n_clusters", "max_iter", "tol"]},
    {"KMedoids": ["n_clusters", "metric_1", "method", "max_iter"]},
    {"DBSCAN": ["eps", "min_samples", "metric_1"]},
    {"HDBSCAN": ["metric_2", "min_samples", "eps"]},
]

# Defining which parameter values each model can take
dict_param_values = {
    "n_clusters": [2, 3, 4, 5, 6, 7, 8, 9, 10],
    "max_iter": [50, 100, 150, 200, 250, 300, 350, 400, 450, 500],
    "tol": [0.00001, 0.0001, 0.001, 0.01, 0.1],
    "metric_1": ["euclidean", "cosine", "haversine", "l2", "cityblock", "l1", "manhattan"],
    "metric_2": ["l2", "canberra", "manhattan", "euclidean", "braycurtis", "chebyshev", "hamming"],
    "method": ["alternate", "pam"],
    "eps": [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 1, 2, 3, 4],
    "min_samples": [3, 4, 5, 6, 7, 8, 9, 10],
}

## Functions for the Genetic Algorithm

In [5]:
# Function to map the model name to instantiating the model
def instantiate_model(model_name):
    if model_name == "KMeans":
        model_out = KMeans()
    elif model_name == "KMedoids":
        model_out = KMedoids()
    elif model_name == "DBSCAN":
        model_out = DBSCAN()
    else:
        model_out = HDBSCAN()
    return model_out

In [6]:
# Defining function to initialize the population
def init_population(
    model_params=list_dict_model_params, param_values=dict_param_values, num_models=15
):
    population = []
    population_models = []

    # Random selection of models and associated parameters to initialize
    for i in range(num_models):
        rand1 = random.randint(0, len(model_params) - 1)
        curr_model_name = list(model_params[rand1].keys())[0]
        curr_model_params = list(model_params[rand1].values())[0]
        params_dict = {}

        # For each parameter, randomly select values from the dictionary
        for param in curr_model_params:
            rand2 = random.randint(0, len(param_values[param]) - 1)
            curr_param_value = param_values[param][rand2]
            if param == "metric_1" or param == "metric_2":
                param = "metric"
            elif param == "eps" and curr_model_name == "HDBSCAN":
                param = "cluster_selection_epsilon"
            else:
                pass
            params_dict[param] = curr_param_value
        population.append({curr_model_name: params_dict})

    # Take the population dictionary and iterate through to create model objects/instantiate
    for i in range(len(population)):
        model_instant = instantiate_model(list(population[i].keys())[0])
        model_instant.set_params(**list(population[i].values())[0])
        population_models.append(model_instant)

    return population_models

In [7]:
# Function to iterate models, fit, and evaluate cluster results.
def cluster_fitness(population, data=df_full_transposed):
    fitness_indices = {}
    for model in population:

        # Try to fit each model and evaluate wtih silhouette score since
        # some model fits may fail due to random nature of instantiating
        try:
            model.fit(data)
            curr_score = silhouette_score(data, model.labels_) 
            fitness_indices[model] = curr_score
        except:
            fitness_indices[model] = -1  
    
    # Sort scoring to get most performant models at the top
    fitness_indices_sorted = {
        k: v
        for k, v in sorted(
            fitness_indices.items(), reverse=True, key=lambda item: item[1] 
        )
    }

    return fitness_indices_sorted

In [17]:
# Function for the selection piece of the algorithm
def population_selection(dict_fitness, selection_param=0.7):
    next_generation = []
    
    # selecting only the top percentage of models based on parameter
    range_val = int(selection_param * len(dict_fitness))
    next_generation = dict(itertools.islice(dict_fitness.items(), range_val))

    return next_generation

In [9]:
# Crossover - If there are models of the same model type (e.g. several HDBSCAN)
# then take random parameters from one and swap it with another and add it
# to the population as a new model. Iterate through the process three times
# to generate three new models.


def crossover(dict_selected_pop, model_params, repeat=3):
    dict_selected_pop_agg = {}
    for model in dict_selected_pop.keys():
        model_name = type(model).__name__
        if model_name in dict_selected_pop_agg.keys():
            dict_selected_pop_agg[model_name].append([model])
        else:
            dict_selected_pop_agg[model_name] = [[model]]

    count = 0
    while count < repeat:
        rand_num = random.randint(0, len(model_params) - 1)
        rand_model = list(model_params[rand_num].keys())[0]
        try:
            len_check = len(dict_selected_pop_agg[rand_model])
        except:
            len_check = 0
        if len_check > 1:
            rand_nums = random.sample(
                range(0, len(dict_selected_pop_agg[rand_model])), 2
            )
            model1 = dict_selected_pop_agg[rand_model][rand_nums[0]][0]
            model2 = dict_selected_pop_agg[rand_model][rand_nums[1]][0]
            half_param_length = int(len(model1.get_params()) / 2)
            model1_param_extract = [{k: v} for k, v in model1.get_params().items()][
                0:half_param_length
            ]
            model2_param_extract = [{k: v} for k, v in model2.get_params().items()][
                half_param_length::
            ]
            model_params_combine = model1_param_extract + model2_param_extract
            dict_model_params_mapping = {
                k: v
                for single_dict in model_params_combine
                for k, v in single_dict.items()
            }
            new_model = instantiate_model(rand_model).set_params(
                **dict_model_params_mapping
            )
            count += 1
            dict_selected_pop[new_model] = 0

    return dict_selected_pop

In [10]:
# Mutation - Select models at random, then select a hyperparameter at random,
# and mutate the parameter to another value in the possible selection list. Iterate
# through twice and generate two new models.


def mutation(dict_selected_pop, model_params, param_values, repeat=2):
    count = 0
    while count < repeat:
        curr_rand_model = random.choice(list(dict_selected_pop.keys()))
        curr_rand_model_name = type(curr_rand_model).__name__
        curr_rand_model_params = curr_rand_model.get_params()
        rand_param_switch = random.choice(list(curr_rand_model_params.keys()))
        if rand_param_switch == "metric" and (
            curr_rand_model_name == "KMedoids" or curr_rand_model_name == "DBSCAN"
        ):
            rand_param_switch = "metric_1"
        elif rand_param_switch == "metric" and curr_rand_model_name == "HDBSCAN":
            rand_param_switch == "metric_2"
        elif rand_param_switch == "cluster_selection_epsilon":
            rand_param_switch == "eps"
        else:
            pass

        try:
            rand_new_value = random.choice(param_values[rand_param_switch])
            flag_raise = 0
        except:
            flag_raise = 1

        if flag_raise == 1:
            pass
        else:
            if rand_param_switch == "metric_1" or rand_param_switch == "metric_2":
                rand_param_switch = "metric"
            elif rand_param_switch == "eps" and curr_rand_model_name == "HDBSCAN":
                rand_param_switch = "cluster_selection_epsilon"
            else:
                pass
            curr_rand_model_params[rand_param_switch] = rand_new_value
            new_model = instantiate_model(curr_rand_model_name).set_params(
                **curr_rand_model_params
            )
            dict_selected_pop[new_model] = 0
            count += 1

    return dict_selected_pop

In [13]:
## Defining the evolution function - combines all previous functions and applies them iteratively

def evolution(model_params, param_values, init_population_num, df, selection_param, crossover_repeat, mutation_repeat, cutoff_score):
    init_pop = init_population(model_params, param_values, init_population_num)
    first_fitness_eval = cluster_fitness(population = init_pop, data = df)
    score = [element for element in first_fitness_eval.values()][0]
    model_hold = {}
    if score < cutoff_score: 
        evaluation_i = first_fitness_eval
    else:
        return [model for model in first_fitness_eval.keys()][0], score
    i = 0
    while score < cutoff_score: 
        if i < 70:
            select_i = population_selection(evaluation_i, selection_param = selection_param)
            crossover_i = crossover(dict_selected_pop = select_i, model_params = model_params, repeat = crossover_repeat)
            mutation_i = mutation(dict_selected_pop = crossover_i, model_params = model_params, param_values = param_values, repeat = mutation_repeat)
            next_population = [model_name for model_name in mutation_i.keys()]
            evaluation_i = cluster_fitness(population = next_population, data = df)
            score = [element for element in evaluation_i.values()][0]
            print(f'Top model is {[model for model in evaluation_i.keys()][0]} and associated score is {score}')
            model_hold[[model for model in evaluation_i.keys()][0]] = score
            i += 1
        else:
            break

    return sorted(model_hold.items(), reverse=True, key=lambda item: item[1])[:10]

In [21]:
evolution(model_params=list_dict_model_params
          , param_values=dict_param_values
          , init_population_num=20
          , df = df_full_transposed
          , selection_param=0.5
          , crossover_repeat=5
          , mutation_repeat=5
          , cutoff_score=0.35)

Top model is KMeans(max_iter=250, n_clusters=10, tol=0.01) and associated score is 0.30248790953532156
Top model is KMeans(max_iter=400, n_clusters=10, tol=0.01) and associated score is 0.32147683039574276
Top model is KMeans(max_iter=250, n_clusters=10, tol=0.01) and associated score is 0.3183130822672847
Top model is KMeans(max_iter=400, n_clusters=10, tol=0.01) and associated score is 0.3226796662985824
Top model is KMeans(max_iter=400, n_clusters=10, tol=0.001) and associated score is 0.32393704954577524
Top model is KMeans(max_iter=400, n_clusters=10, tol=1e-05) and associated score is 0.3226796662985824
Top model is KMeans(max_iter=400, n_clusters=10, tol=0.01) and associated score is 0.32147683039574276
Top model is KMeans(max_iter=400, n_clusters=10, tol=0.1) and associated score is 0.3235276820505186
Top model is KMeans(max_iter=250, n_clusters=10, tol=1e-05) and associated score is 0.3184300308731838
Top model is KMeans(max_iter=400, n_clusters=10, tol=0.01) and associated sc

[(KMeans(max_iter=400, n_clusters=10, tol=0.001), 0.32393704954577524),
 (KMeans(max_iter=350, n_clusters=10, tol=0.1), 0.32393704954577524),
 (KMeans(max_iter=400, n_clusters=10, tol=0.1), 0.3235276820505186),
 (KMeans(max_iter=400, n_clusters=10, tol=0.01), 0.3235276820505186),
 (KMeans(max_iter=100, n_clusters=10), 0.3235276820505186),
 (KMeans(n_clusters=10, tol=1e-05), 0.3235276820505186),
 (KMeans(max_iter=350, n_clusters=10, tol=0.1), 0.3235276820505186),
 (KMeans(max_iter=350, n_clusters=10, tol=1e-05), 0.3235276820505186),
 (KMeans(max_iter=350, n_clusters=10, tol=0.1), 0.3235276820505186),
 (KMeans(n_clusters=10, tol=1e-05), 0.3235276820505186)]